# Geolocalisation des arbres

In [1]:
import pandas as pd 
import numpy as np
import folium

In [2]:
df = pd.read_csv('data/trees.csv', converters={'GeoJSON': pd.read_json})

## Data Cleaning

In [56]:
df_geo = df[['GENRE_BOTA', 'ESPECE', 'VARIETE', 
             'STADEDEDEVELOPPEMENT', 'ANNEEDEPLANTATION', 
             'latitude', 'longitude']].copy()

gps_series = df.loc[:,'GeoJSON'].map(lambda x: tuple(x.coordinates))
df['LATITUDE'] = gps_series.str.get(0)
df['LONGITUDE'] = gps_series.str.get(1)

df_geo.fillna({'GENRE_BOTA' : '[Genre inconnu]',
               'ESPECE' : '[Espèce inconnue]',
               'VARIETE' : ' '}, 
              inplace=True)

df_geo.rename(columns={'GENRE_BOTA' : 'genre',
                      'STADEDEDEVELOPPEMENT' : 'stade_de_developpement',
                      'ANNEEDEPLANTATION' : 'annee_de_plantation'}, inplace=True)

df_geo.rename(columns=str.title, inplace=True)

In [55]:
df_geo.sample(n=5)

,Genre,Espece,Variete,Stade_De_Developpement,Annee_De_Plantation,Latitude,Longitude
13871,Malus,domestica,,Arbre jeune,2014.0,5.710423,45.191575
2276,Acer,pseudoplatanus,,Arbre adulte,1970.0,5.739528,45.189859
260,Sophora,japonica,,Arbre adulte,1970.0,5.705256,45.166213
14363,Liquidambar,styraciflua,,Arbre adulte,1987.0,5.736069,45.169628
24299,Pinus,nigra,Austriaca,Arbre adulte,1975.0,5.736624,45.164640


## On dessine des cartes !

In [ ]:
gps_points = data.map(lambda x: (x.coordinates[0], x.coordinates[1]))
df_geo['gpsCoord'] = gps_points

In [48]:
m = folium.Map(
    location=[45.17798, 5.74564], 
    zoom_start=13
)

def addToMap(x):
    folium.Marker(
        [x['gpsCoord'][1], x['gpsCoord'][0]], 
        popup=f'<i>{x["GENRE_BOTA"]}</i>', 
        tooltip=''.join([x["GENRE_BOTA"], ' ', x["ESPECE"]]),
        icon=folium.Icon(color='green', icon='leaf')
    ).add_to(m)

# df_geo[df_geo['ANNEEDEPLANTATION']==1971].apply(addToMap, axis=1)

m

## Heatmap de l'ensemble des arbres du dataset

In [ ]:
from folium import plugins

m = folium.Map(
    location=[45.17798, 5.74564], 
    tiles='stamentoner',
    zoom_start=13
)

latitude = df_geo['gpsCoord'].map(lambda x: x[1]).values
longitude = df_geo['gpsCoord'].map(lambda x: x[0]).values
trees_coord = np.stack((latitude, longitude), axis=0).T

m.add_child(plugins.HeatMap(trees_coord, radius=8))

m

## Arbres vieillissants à Grenoble

In [ ]:
m = folium.Map(
    location=[45.17798, 5.74564], 
    tiles='stamentoner',
    zoom_start=13
)

latitude = df_geo[df_geo["STADEDEDEVELOPPEMENT"] == 'Arbre vieillissant']['gpsCoord'].map(lambda x: x[1]).values
longitude = df_geo[df_geo["STADEDEDEVELOPPEMENT"] == 'Arbre vieillissant']['gpsCoord'].map(lambda x: x[0]).values
trees_coord = np.stack((latitude, longitude), axis=0).T

m.add_child(plugins.HeatMap(trees_coord, radius=15))

m

## Distribution des platanes 
### Conclusion => le bitume fait pousser des platanes

In [ ]:
m = folium.Map(
    location=[45.17798, 5.74564], 
    tiles='stamentoner',
    zoom_start=13
)

latitude = df_geo[df_geo["GENRE_BOTA"] == 'Platanus']['gpsCoord'].map(lambda x: x[1]).values
longitude = df_geo[df_geo["GENRE_BOTA"] == 'Platanus']['gpsCoord'].map(lambda x: x[0]).values
trees_coord = np.stack((latitude, longitude), axis=0).T

m.add_child(plugins.HeatMap(trees_coord, radius=12))

m

In [ ]:
import folium
m = folium.Map(location=[48.8274, 2.31144], tiles='stamentoner')

m

## Test appel API ArcGIS -> Ne pas faire trop souvent 

In [ ]:
from geopy.geocoders import ArcGIS, Nominatim
from geopy.extra.rate_limiter import RateLimiter

geolocator = ArcGIS(user_agent="plouf")
data = df_geo['GeoJSON'][0:10].map(pd.read_json)
gps_coord = data.map(lambda x: ''.join([str(x.coordinates[1]), ', ', str(x.coordinates[0])])) 
# 1 -delay geocoding calls pour pas se faire engueuler
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=0.2)
gps_coord.map(geolocator.reverse)